## Card Game Suppress Seven

### Rules
    - Use the 52 cards in 4 colors: Heart, Diamond, Spade and Club.
    - Four players, each one gets 13 cards in the beginning.
    - The player who has Spade 7 on hand goes first.
    - The players must use the cards which have the same color and consecutive number to the cards on the board.
        - For example, if there is ♠7 on board, the next player must use ♠6, ♠8 or ♥7,♦7,♣7.
    - If a player has not card can be used, he/she needs to suppress one card. This card should not be seen by the other players until the end of this game.
    - The total points of the suppressed cards would be calculated at the end of the game, the one has the least points win.

In [1]:
import numpy as np
import names
import random
from collections import OrderedDict, deque, defaultdict

In [2]:
class CustomError(Exception):
    """Base class for exceptions in this module."""
    pass

In [7]:
class CardNumberError(CustomError):
    def __init__(self, message, number):
        self.message = message
        self.number = number
    def __str__(self):
        return f"The numer {self.number} is {self.message} than the allowed card number range: 1~52."
class CardNumberDisaster(CustomError):
    def __init__(self,card):
        self.card = card
    def __str__(self):
        return f"The card numer {self.card.name()} is illegal."

In [40]:
# It is the card class, there are 52 cards in total, 13 cards for each suit. 
# So that we use the id between 1 and 52 to initialize the card.
class Card:
    suit_map = {0: 'Heart',
                1: 'Diamond',
                2: 'Spade',
                3: 'Club'}
    def __init__(self, number, card_code=None):
        if number > 52:
            raise CardNumberError('larger', number)
        elif number < 1:
            raise CardNumberError('smaller', number)
        else:
            self.number = (number-1)%13+1     # it is the number of this card, no matter which suit, 1-13
            self.suit = Card.suit_map[(number-1)//13] # it is the suit, one of the four
            self.card_id = number              # it is the id of the card, each card has an unique id, 1-52
            self.name = self.name()
            self.left = None
            self.right = None
            self.previous = None
            self.available = False
            self.card_code = number if card_code is None else card_code
    def __str__(self):
        return f"This card is {self.suit} {self.number}. The uid of this card is {self.card_id}."
    
    def name(self):
        return f"{self.suit} {self.number}"

In [41]:
class Player:
    identity=0    # it will be accumlated for each player
    def __init__(self, name=None, email=None):
        Player.identity += 1
        self.name = name or names.get_first_name()
        self.email = email
        self.player_id = Player.identity
        self.hand_card = None
        self.decoder = {} # only the player himself/herself knows the code of his/her own cards
    
    def get_hand_cards(self, hand_card):
        for card in hand_card:
            card.player_id = self.player_id
            self.decoder[card.card_code] = card.card_id # All the card are represented by card id in the backend
        self.hand_card = OrderedDict(sorted([(card.card_id, card) for card in hand_card]))

In [42]:
class Game:
    identity=0
    def __init__(self, random_seed=123):
        Game.identity += 1
        self.game_id = Game.identity
        self.card_map = OrderedDict()    # all the calculation in the backend uses the id (card or player), 
        self.player_map = OrderedDict()  # need to use this map to link the id to the object
        self.card_pool = []
        self.players = deque([])   # Here the deque is being used, because the player with Spade 7 need to start, 
                                   # NOT based on the order of geting card
        pool = list(range(1,53))
        random.seed(random_seed)
        random.shuffle(pool)
        self.encoder = {}          # create the visiable code for each card, it is different from the card id, 
                                   #since we do not want the other players know which card is suppressed. 
                                   # It is different in different game.
        for i, v in enumerate(pool):
            self.encoder[i+1] = v
    def get_cards(self):
        previous_card = None
        """
        left means the card which number is -1 of current one, right means +1 of current card.
        However, for card.number < 7, +1 card is the previous card of current one, means if the previous card is not on the board, 
        the current one cannot be available. Since the +1 card already stored as the previous card, no need to set the right card.
        The same for card > 7, no need to set .left.
        """
        for i in range(1,53):
            this_card = Card(i, self.encoder[i])
            if this_card.number < 7:             
                this_card.left = previous_card    # left means the card which number is one less than the current one
                if previous_card is not None:
                    previous_card.previous = this_card  # previous card means, if previous one is not on the board, 
                                                        # this card cannot be used
            elif this_card.number > 7:
                this_card.previous = previous_card     #7 is the previous for both 6 and 8, here sets 8's.
                if previous_card is not None:
                    previous_card.right = this_card 
            elif this_card.number == 7:           # For all suits, 7 is always be available, and it is the previous for both 6 and 8, 
                                                  # here sets 6's. 
                this_card.left = previous_card
                previous_card.previous = this_card
                this_card.available = True      # set 7 to available

            else:
                raise CardNumberDisaster(this_card)  # not going to happen
            if this_card.number == 13:
                previous_card = None
            else:
                previous_card = this_card
            self.card_map[this_card.card_id] = this_card
        self.card_pool = list(self.card_map.values())   # list of card class objs
        random.shuffle(self.card_pool)
    
    def prepare(self):
        for i in range(4):     # make sure only four players
            this_name = input("Please input your name: ")
            this_name = None if this_name == '' else this_name
            this_email = input("Please input your email: ")
            this_email = None if this_email == '' else this_email
            this_player = Player(this_name, this_email)
            print(f"Welcome {this_player.name}, have fun!")
            this_player.get_hand_cards(self.card_pool[i*13:(i+1)*13])
            self.player_map[this_player.player_id] = this_player
            self.players.append(this_player)  # Here the players are added to the deque one by one
            
        init_player = self.player_map[self.card_map[33].player_id]  # The one has Spade 7 need to be the first player
        first_player_index = self.players.index(init_player)
        # Here the new order is rotated, the same order as assigning cards, but the first player has the Spade 7
        self.players.rotate(-1*first_player_index)

In [56]:
first_game = Game()

In [57]:
first_game.get_cards()

In [58]:
first_game.card_map

OrderedDict([(1, <__main__.Card at 0x28cffe930a0>),
             (2, <__main__.Card at 0x28cffdeadf0>),
             (3, <__main__.Card at 0x28cffdea880>),
             (4, <__main__.Card at 0x28cffdea5b0>),
             (5, <__main__.Card at 0x28cffded700>),
             (6, <__main__.Card at 0x28cffded640>),
             (7, <__main__.Card at 0x28cffded730>),
             (8, <__main__.Card at 0x28cffded1f0>),
             (9, <__main__.Card at 0x28cffded910>),
             (10, <__main__.Card at 0x28cffded550>),
             (11, <__main__.Card at 0x28cffded5e0>),
             (12, <__main__.Card at 0x28cffdeddc0>),
             (13, <__main__.Card at 0x28cffded340>),
             (14, <__main__.Card at 0x28cffdedb50>),
             (15, <__main__.Card at 0x28cffdedac0>),
             (16, <__main__.Card at 0x28cffded9a0>),
             (17, <__main__.Card at 0x28cffdedfa0>),
             (18, <__main__.Card at 0x28cffded4f0>),
             (19, <__main__.Card at 0x28cffded8e0>),
  

In [59]:
first_game.prepare()

Please input your name: 
Please input your email: 
Welcome Ashley, have fun!
Please input your name: 
Please input your email: 
Welcome Linda, have fun!
Please input your name: 
Please input your email: 
Welcome Andrew, have fun!
Please input your name: 
Please input your email: 
Welcome Thomas, have fun!


In [60]:
first_game.players[0].name

'Thomas'

In [62]:
first_game.players[0].hand_card

OrderedDict([(1, <__main__.Card at 0x28cffe930a0>),
             (26, <__main__.Card at 0x28cffded220>),
             (28, <__main__.Card at 0x28cffdedd90>),
             (32, <__main__.Card at 0x28cffded9d0>),
             (33, <__main__.Card at 0x28cffded4c0>),
             (34, <__main__.Card at 0x28cffdedca0>),
             (35, <__main__.Card at 0x28cffded8b0>),
             (39, <__main__.Card at 0x28cffdd3070>),
             (41, <__main__.Card at 0x28cffdd38b0>),
             (42, <__main__.Card at 0x28cffdd3550>),
             (43, <__main__.Card at 0x28cffdd30d0>),
             (44, <__main__.Card at 0x28cffdd3fd0>),
             (50, <__main__.Card at 0x28cffdd3520>)])

In [65]:
for card in first_game.players[0].hand_card.values():
    print(card)
    if card.available:
        print('it is available')

This card is Heart 1. The uid of this card is 1.
This card is Diamond 13. The uid of this card is 26.
This card is Spade 2. The uid of this card is 28.
This card is Spade 6. The uid of this card is 32.
This card is Spade 7. The uid of this card is 33.
it is available
This card is Spade 8. The uid of this card is 34.
This card is Spade 9. The uid of this card is 35.
This card is Spade 13. The uid of this card is 39.
This card is Club 2. The uid of this card is 41.
This card is Club 3. The uid of this card is 42.
This card is Club 4. The uid of this card is 43.
This card is Club 5. The uid of this card is 44.
This card is Club 11. The uid of this card is 50.


In [54]:
first_game.players[0].hand_card[33].name

'Spade 7'

In [55]:
first_game.players[0].decoder

{16: 34,
 1: 32,
 11: 39,
 36: 43,
 42: 41,
 37: 35,
 35: 44,
 46: 28,
 39: 33,
 22: 42,
 6: 50,
 13: 26,
 38: 1}

In [30]:
print(first_game.card_map[33])

This card is Spade 7. The uid of this card is 33.


In [17]:
first_game.card_map[33].previous

In [20]:
first_game.card_map[32].name

'Spade 6'

In [19]:
first_game.card_map[32].previous.name

'Spade 7'

In [21]:
first_game.card_map[34].name

'Spade 8'

In [22]:
first_game.card_map[34].previous.name

'Spade 7'

In [18]:
# test
test_card = Card(60)

CardNumberError: The numer 60 is larger than the allowed card number range: 1~52.

In [8]:
test_card

In [10]:
test_card.name()

'Diamond 7'

In [12]:
print(test_card)

This card is Diamond 7. The uid of this card is 20.
